In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta
from pandas import json_normalize

In [45]:
# Definir los parámetros iniciales
url = "https://toteatglobal.appspot.com/mw/or/1.0/sales"
params = {
    "xir": "6502106216529920",  # ID de restaurante
    "xil": "2",                  # ID de local
    "xiu": "1002",               # ID de usuario
    "xapitoken": "pedir_token"  # Token de API
}

# Definir el rango de fechas inicial y final
fecha_inicio = datetime(2021, 8, 13)
fecha_fin = datetime.now()

# Lista para almacenar los DataFrames de cada segmento de fechas
dfs = []

# Iterar sobre los segmentos de fechas
while fecha_inicio <= fecha_fin:
    # Calcular la fecha final del segmento (15 días después)
    fecha_segmento_fin = fecha_inicio + timedelta(days=14)
    if fecha_segmento_fin > fecha_fin:
        fecha_segmento_fin = fecha_fin
    
    # Actualizar los parámetros con las fechas del segmento
    params["ini"] = fecha_inicio.strftime("%Y%m%d")
    params["end"] = fecha_segmento_fin.strftime("%Y%m%d")
    
    # Realizar la solicitud GET
    response = requests.get(url, params=params)
    
    # Verificar si la solicitud fue exitosa
    if response.status_code == 200:
        data = response.json()
        
        # Convertir los datos a un DataFrame de pandas
        df = pd.DataFrame(data['data'])
        
        # Eliminar las columnas no deseadas
        columns_to_drop = ['comment', 'numberClients', 'registerId', 'totalWithGratuity', 'registerName',
                           'fiscalAmt', 'fiscalType', 'zoneName',
                           'difference', 'change', 'client',
                           'fiscalId', 'fiscalPrinter']
        df.drop(columns_to_drop, axis=1, inplace=True)
        
        # Agregar el DataFrame modificado a la lista
        dfs.append(df)
    else:
        print("Error en la solicitud:", response.status_code)
    
    # Actualizar la fecha de inicio para el próximo segmento
    fecha_inicio = fecha_segmento_fin + timedelta(days=1)

# Concatenar todos los DataFrames en uno solo
ventas_df = pd.concat(dfs, ignore_index=True)

# Mostrar el DataFrame resultante
print(ventas_df)


      tableName    total  waiterId  gratuity  zoneId  discounts  \
0       Virtual   6990.0         0       0.0                0.0   
1           T16  17970.0         3       0.0  ZB.010        0.0   
2            T7   6990.0         3       0.0  ZB.010        0.0   
3           T14  19980.0         3       0.0  ZB.010    -3700.0   
4            T3  12880.0         3       0.0  ZB.010        0.0   
...         ...      ...       ...       ...     ...        ...   
53599       AF6  54930.0        19    5493.0  ZB.010        0.0   
53600       AF3  86780.0        19    8678.0  ZB.010        0.0   
53601       T10  26170.0        19    2617.0  ZB.010        0.0   
53602   Virtual  21990.0         0       0.0                0.0   
53603       AF1  26370.0        16    2636.0  ZB.010        0.0   

                  dateOpen                                           products  \
0      2021-08-14T00:55:44  [{'payed': 6990.0, 'discounts': 0, 'hierarchyN...   
1      2021-08-14T01:26:35  [{'pa

In [46]:
# Tabla de ventas historicas, sin limpiar.
ventas_df

,tableName,total,waiterId,gratuity,zoneId,discounts,dateOpen,products,orderId,paymentForms,waiterName,subtotal,tableCapaticy,taxes,payed,tableId,dateClosed,paymentId
0,Virtual,6990.0,0,0.0,,0.0,2021-08-14T00:55:44,"[{'payed': 6990.0, 'discounts': 0, 'hierarchyN...",4617000165572608,"[{'amount': 10203.0, 'name': 'Web Pay', 'id': ...",,5361.0,,1629.0,10203.0,V-90002,2021-08-14T01:50:47,1628902544986563
1,T16,17970.0,3,0.0,ZB.010,0.0,2021-08-14T01:26:35,"[{'payed': 5990.0, 'discounts': 0, 'hierarchyN...",4878437089804288,"[{'amount': 17970, 'name': 'PedidosYa Efectivo...",Pedro Rodriguez,15102.0,4,2868.0,17970.0,18,2021-08-14T20:23:29,1628904372492113
2,T7,6990.0,3,0.0,ZB.010,0.0,2021-08-14T01:33:55,"[{'payed': 6990.0, 'discounts': 0, 'hierarchyN...",5124727694426112,"[{'amount': 6990, 'name': 'Efectivo', 'id': 10...",Pedro Rodriguez,5874.0,4,1116.0,6990.0,7,2021-08-14T01:42:23,1628904829769503
3,T14,19980.0,3,0.0,ZB.010,-3700.0,2021-08-13T21:10:13,"[{'payed': 11399.0, 'discounts': -2591, 'hiera...",5246704765632512,"[{'amount': 16280, 'name': 'Uber eats', 'id': ...",Pedro Rodriguez,17381.0,4,2599.0,16280.0,16,2021-08-13T21:26:44,1628888977318853
4,T3,12880.0,3,0.0,ZB.010,0.0,2021-08-13T22:25:19,"[{'payed': 6290.0, 'discounts': 0, 'hierarchyN...",5308026161201152,"[{'amount': 12880, 'name': 'Efectivo', 'id': 1...",Pedro Rodriguez,10824.0,4,2056.0,12880.0,3,2021-08-14T00:04:35,1628893486702303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53599,AF6,54930.0,19,5493.0,ZB.010,0.0,2024-02-10T23:27:32,"[{'payed': 8990.0, 'discounts': 0, 'hierarchyN...",6643010967306240,"[{'amount': 60423, 'name': 'Tarjeta Débito', '...",Ronny Vegas,46161.0,4,8769.0,60423.0,39,2024-02-11T05:34:53,1707607617448419
53600,AF3,86780.0,19,8678.0,ZB.010,0.0,2024-02-11T02:59:49,"[{'payed': 0.0, 'discounts': 0, 'hierarchyName...",6683520360644608,"[{'amount': 95458, 'name': 'Tarjeta Débito', '...",Ronny Vegas,72925.0,4,13855.0,95458.0,36,2024-02-11T05:36:28,1707620370375219
53601,T10,26170.0,19,2617.0,ZB.010,0.0,2024-02-11T02:13:53,"[{'payed': 0.0, 'discounts': 0, 'hierarchyName...",6739510057631744,"[{'amount': 28787, 'name': 'Efectivo', 'id': 1...",Ronny Vegas,21992.0,4,4178.0,28787.0,10,2024-02-11T05:27:24,1707617626098519
53602,Virtual,21990.0,0,0.0,,0.0,2024-02-11T01:52:18,"[{'payed': 21990.0, 'discounts': 0, 'hierarchy...",6740702414372864,"[{'amount': 21990, 'name': 'UberEats', 'id': 5...",,18479.0,,3511.0,21990.0,V-90014,2024-02-11T01:57:04,1707616338466286


In [48]:
# Se guarda para ver podibles cambios
ventas_df.to_csv("./archivos_csv/ventas_historico.csv", index=False)

In [2]:
ventas_df = pd.read_csv("./archivos_csv/ventas_historico.csv")
ventas_df

,tableName,total,waiterId,gratuity,zoneId,discounts,dateOpen,products,orderId,paymentForms,waiterName,subtotal,tableCapaticy,taxes,payed,tableId,dateClosed,paymentId
0,Virtual,6990.0,0,0.0,NaN,0.0,2021-08-14T00:55:44,"[{'payed': 6990.0, 'discounts': 0, 'hierarchyN...",4617000165572608,"[{'amount': 10203.0, 'name': 'Web Pay', 'id': ...",NaN,5361.0,NaN,1629.0,10203.0,V-90002,2021-08-14T01:50:47,1628902544986563
1,T16,17970.0,3,0.0,ZB.010,0.0,2021-08-14T01:26:35,"[{'payed': 5990.0, 'discounts': 0, 'hierarchyN...",4878437089804288,"[{'amount': 17970, 'name': 'PedidosYa Efectivo...",Pedro Rodriguez,15102.0,4.0,2868.0,17970.0,18,2021-08-14T20:23:29,1628904372492113
2,T7,6990.0,3,0.0,ZB.010,0.0,2021-08-14T01:33:55,"[{'payed': 6990.0, 'discounts': 0, 'hierarchyN...",5124727694426112,"[{'amount': 6990, 'name': 'Efectivo', 'id': 10...",Pedro Rodriguez,5874.0,4.0,1116.0,6990.0,7,2021-08-14T01:42:23,1628904829769503
3,T14,19980.0,3,0.0,ZB.010,-3700.0,2021-08-13T21:10:13,"[{'payed': 11399.0, 'discounts': -2591, 'hiera...",5246704765632512,"[{'amount': 16280, 'name': 'Uber eats', 'id': ...",Pedro Rodriguez,17381.0,4.0,2599.0,16280.0,16,2021-08-13T21:26:44,1628888977318853
4,T3,12880.0,3,0.0,ZB.010,0.0,2021-08-13T22:25:19,"[{'payed': 6290.0, 'discounts': 0, 'hierarchyN...",5308026161201152,"[{'amount': 12880, 'name': 'Efectivo', 'id': 1...",Pedro Rodriguez,10824.0,4.0,2056.0,12880.0,3,2021-08-14T00:04:35,1628893486702303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53599,AF6,54930.0,19,5493.0,ZB.010,0.0,2024-02-10T23:27:32,"[{'payed': 8990.0, 'discounts': 0, 'hierarchyN...",6643010967306240,"[{'amount': 60423, 'name': 'Tarjeta Débito', '...",Ronny Vegas,46161.0,4.0,8769.0,60423.0,39,2024-02-11T05:34:53,1707607617448419
53600,AF3,86780.0,19,8678.0,ZB.010,0.0,2024-02-11T02:59:49,"[{'payed': 0.0, 'discounts': 0, 'hierarchyName...",6683520360644608,"[{'amount': 95458, 'name': 'Tarjeta Débito', '...",Ronny Vegas,72925.0,4.0,13855.0,95458.0,36,2024-02-11T05:36:28,1707620370375219
53601,T10,26170.0,19,2617.0,ZB.010,0.0,2024-02-11T02:13:53,"[{'payed': 0.0, 'discounts': 0, 'hierarchyName...",6739510057631744,"[{'amount': 28787, 'name': 'Efectivo', 'id': 1...",Ronny Vegas,21992.0,4.0,4178.0,28787.0,10,2024-02-11T05:27:24,1707617626098519
53602,Virtual,21990.0,0,0.0,NaN,0.0,2024-02-11T01:52:18,"[{'payed': 21990.0, 'discounts': 0, 'hierarchy...",6740702414372864,"[{'amount': 21990, 'name': 'UberEats', 'id': 5...",NaN,18479.0,NaN,3511.0,21990.0,V-90014,2024-02-11T01:57:04,1707616338466286


In [3]:
copia = ventas_df.copy()

In [4]:
# Hacer una tabla solo de ventas por productos
ventasxproducts_parcial = copia[["orderId", "products"]]
print(ventasxproducts_parcial)

                orderId                                           products
0      4617000165572608  [{'payed': 6990.0, 'discounts': 0, 'hierarchyN...
1      4878437089804288  [{'payed': 5990.0, 'discounts': 0, 'hierarchyN...
2      5124727694426112  [{'payed': 6990.0, 'discounts': 0, 'hierarchyN...
3      5246704765632512  [{'payed': 11399.0, 'discounts': -2591, 'hiera...
4      5308026161201152  [{'payed': 6290.0, 'discounts': 0, 'hierarchyN...
...                 ...                                                ...
53599  6643010967306240  [{'payed': 8990.0, 'discounts': 0, 'hierarchyN...
53600  6683520360644608  [{'payed': 0.0, 'discounts': 0, 'hierarchyName...
53601  6739510057631744  [{'payed': 0.0, 'discounts': 0, 'hierarchyName...
53602  6740702414372864  [{'payed': 21990.0, 'discounts': 0, 'hierarchy...
53603  6750384411508736  [{'payed': 0.0, 'discounts': 0, 'hierarchyName...

[53604 rows x 2 columns]


In [5]:
ventasxproducts_parcial.drop_duplicates(subset=['orderId'], inplace=True)
ventasxproducts_parcial.shape

C:\Users\matia\AppData\Local\Temp\ipykernel_34572\1821769510.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ventasxproducts_parcial.drop_duplicates(subset=['orderId'], inplace=True)


(53380, 2)

In [6]:
ventasxproducts_parcial['products'] = ventasxproducts_parcial['products'].apply(lambda x: eval(x))

expanded_rows = []
# Iterar sobre cada fila del DataFrame original
for index, row in ventasxproducts_parcial.iterrows():
    order_id = row['orderId']
    products = row['products']
    # Iterar sobre cada producto en la lista de productos
    for product in products:
        # Crear un nuevo diccionario que representa una fila expandida
        expanded_row = {**product, 'orderId': order_id}
        expanded_rows.append(expanded_row)

# Crear un nuevo DataFrame con las filas expandidas
df_expanded = pd.DataFrame(expanded_rows)

df_expanded

C:\Users\matia\AppData\Local\Temp\ipykernel_34572\2474126212.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ventasxproducts_parcial['products'] = ventasxproducts_parcial['products'].apply(lambda x: eval(x))


,payed,discounts,hierarchyName,netPrice,name,hierarchyId,id,taxes,quantity,orderId
0,6990.0,0.0,Burgers🍔,6990.0,Hamburguesa Americana🥓,AB.410,9050,1116.0,1,4617000165572608
1,3213.0,0.0,Todos los Productos,3213.0,Costo Delivery,AB.,TOTEATDVYCOST,513.0,1,4617000165572608
2,5990.0,0.0,Fries,5990.0,Mexa Grande,AB.010,1090,956.0,1,4878437089804288
3,5990.0,0.0,Fries,5990.0,Cheese and bacon Grande,AB.010,1050,956.0,1,4878437089804288
4,5990.0,0.0,Fries,5990.0,Pollo albahaca grande,AB.010,1920,956.0,1,4878437089804288
...,...,...,...,...,...,...,...,...,...,...
223409,0.0,0.0,Elige tus papas grandes,0.0,Mexa grande eats,BA.330,2340,0.0,1,6740702414372864
223410,0.0,0.0,Bar,0.0,Mojitos,AB.075,3480,0.0,1,6750384411508736
223411,13380.0,0.0,Bar,6690.0,Mojito coco,AB.075,7450,2136.0,2,6750384411508736
223412,0.0,0.0,PAPITAS,0.0,Cheese & bacon,AB.530,4370,0.0,1,6750384411508736


In [21]:
# esta la usamos mas adelante
para_products_y_hierarchy = df_expanded.copy()

In [22]:
columna_a_mover = df_expanded.pop('orderId')
df_expanded.insert(0, 'orderId', columna_a_mover)
df_expanded.head(2)

,orderId,payed,discounts,hierarchyName,netPrice,name,hierarchyId,id,taxes,quantity
0,4617000165572608,6990.0,0.0,Burgers🍔,6990.0,Hamburguesa Americana🥓,AB.410,9050,1116.0,1
1,4617000165572608,3213.0,0.0,Todos los Productos,3213.0,Costo Delivery,AB.,TOTEATDVYCOST,513.0,1


In [23]:
df_expanded['discounts'].unique()

array([    0., -2591., -1109., ..., -3582., -3583., -5825.])

In [24]:
# tabla final
ventasxproducts = df_expanded[['orderId', 'payed', 'id', 'taxes', 'quantity', 'discounts']]
ventasxproducts

,orderId,payed,id,taxes,quantity,discounts
0,4617000165572608,6990.0,9050,1116.0,1,0.0
1,4617000165572608,3213.0,TOTEATDVYCOST,513.0,1,0.0
2,4878437089804288,5990.0,1090,956.0,1,0.0
3,4878437089804288,5990.0,1050,956.0,1,0.0
4,4878437089804288,5990.0,1920,956.0,1,0.0
...,...,...,...,...,...,...
223409,6740702414372864,0.0,2340,0.0,1,0.0
223410,6750384411508736,0.0,3480,0.0,1,0.0
223411,6750384411508736,13380.0,7450,2136.0,2,0.0
223412,6750384411508736,0.0,4370,0.0,1,0.0


In [25]:
ventasxproducts = ventasxproducts[['orderId', 'id', 'quantity', 'payed', 'taxes', 'discounts']]
ventasxproducts.rename(columns={'orderId': 'order_id', 'id': 'product_id'}, inplace=True)
ventasxproducts.head()

,order_id,product_id,quantity,payed,taxes,discounts
0,4617000165572608,9050,1,6990.0,1116.0,0.0
1,4617000165572608,TOTEATDVYCOST,1,3213.0,513.0,0.0
2,4878437089804288,1090,1,5990.0,956.0,0.0
3,4878437089804288,1050,1,5990.0,956.0,0.0
4,4878437089804288,1920,1,5990.0,956.0,0.0


In [26]:
ventasxproducts.to_csv("./archivos_csv/ventasxproducts.csv", index=False)

In [109]:
# tabla mesas
mesas_parcial = copia[['tableName', 'tableId', 'tableCapaticy']]
mesas_parcial

,tableName,tableId,tableCapaticy
0,Virtual,V-90002,NaN
1,T16,18,4.0
2,T7,7,4.0
3,T14,16,4.0
4,T3,3,4.0
...,...,...,...
53599,AF6,39,4.0
53600,AF3,36,4.0
53601,T10,10,4.0
53602,Virtual,V-90014,NaN


In [110]:
mesas = mesas_parcial.drop_duplicates(subset=['tableId']).reset_index(drop=True)
mesas = mesas[['tableId', 'tableName', 'tableCapaticy']]
mesas

,tableId,tableName,tableCapaticy
0,V-90002,Virtual,NaN
1,18,T16,4.0
2,7,T7,4.0
3,16,T14,4.0
4,3,T3,4.0
...,...,...,...
206,45,AF12,4.0
207,49,AF16,4.0
208,46,AF13,4.0
209,47,AF14,4.0


In [112]:
mesas.rename(columns={'tableId': 'id', 'tableName': 'name', 'tableCapaticy': 'capacity'}, inplace=True)
mesas.fillna(0, inplace=True)
mesas['capacity'] = mesas['capacity'].astype(int)
mesas.head()

,id,name,capacity
0,V-90002,Virtual,0
1,18,T16,4
2,7,T7,4
3,16,T14,4
4,3,T3,4


In [113]:
mesas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        211 non-null    object
 1   name      211 non-null    object
 2   capacity  211 non-null    int32 
dtypes: int32(1), object(2)
memory usage: 4.2+ KB


In [114]:
mesas.to_csv("./archivos_csv/mesas.csv", index=False)

In [182]:
# tabla meseros
meseros_parcial = copia[['waiterId', 'waiterName']]
meseros_parcial

,waiterId,waiterName
0,0,NaN
1,3,Pedro Rodriguez
2,3,Pedro Rodriguez
3,3,Pedro Rodriguez
4,3,Pedro Rodriguez
...,...,...
53599,19,Ronny Vegas
53600,19,Ronny Vegas
53601,19,Ronny Vegas
53602,0,NaN


In [183]:
meseros = meseros_parcial.drop_duplicates(subset=['waiterId']).reset_index(drop=True)
meseros['waiterName'].fillna("Sin Especificar", inplace=True)
meseros.rename(columns={'waiterId': 'id', 'waiterName': 'name'}, inplace=True)
meseros

,id,name
0,0,Sin Especificar
1,3,Pedro Rodriguez
2,1,Leslie Canelón
3,20,Frank
4,6649939665616896,Sin Especificar
5,5133770745970688,Sin Especificar
6,6406847628574720,Sin Especificar
7,6301468951576576,Sin Especificar
8,6312141743980544,Sin Especificar
9,5140254229790720,Sin Especificar


In [184]:
meseros.to_csv("./archivos_csv/meseros.csv", index=False)

In [28]:
# tabla forma_pago
forma_pago_parcial = copia[['orderId', 'paymentForms']]
forma_pago_parcial.drop_duplicates(subset=['orderId'], inplace=True)
forma_pago_parcial.shape

C:\Users\matia\AppData\Local\Temp\ipykernel_20040\136341455.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forma_pago_parcial.drop_duplicates(subset=['orderId'], inplace=True)


(53380, 2)

In [29]:
forma_pago_parcial['paymentForms'] = forma_pago_parcial['paymentForms'].apply(lambda x: eval(x))

expanded_rows = []
# Iterar sobre cada fila del DataFrame original
for index, row in forma_pago_parcial.iterrows():
    order_id = row['orderId']
    payments = row['paymentForms']
    # Iterar sobre cada producto en la lista de productos
    for payment in payments:
        # Crear un nuevo diccionario que representa una fila expandida
        expanded_row = {**payment, 'orderId': order_id}
        expanded_rows.append(expanded_row)

# Crear un nuevo DataFrame con las filas expandidas
df_expanded = pd.DataFrame(expanded_rows)

df_expanded

C:\Users\matia\AppData\Local\Temp\ipykernel_20040\2581911872.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forma_pago_parcial['paymentForms'] = forma_pago_parcial['paymentForms'].apply(lambda x: eval(x))


,amount,name,id,orderId
0,10203.0,Web Pay,5000,4617000165572608
1,17970.0,PedidosYa Efectivo,50002,4878437089804288
2,6990.0,Efectivo,1000,5124727694426112
3,16280.0,Uber eats,50004,5246704765632512
4,12880.0,Efectivo,1000,5308026161201152
...,...,...,...,...
54720,95458.0,Tarjeta Débito,3000,6683520360644608
54721,28787.0,Efectivo,1000,6739510057631744
54722,21990.0,UberEats,5103,6740702414372864
54723,14503.0,Tarjeta Débito,3000,6750384411508736


In [30]:
df_expanded.drop("name", axis=1, inplace=True)
df_expanded.head(2)

,amount,id,orderId
0,10203.0,5000,4617000165572608
1,17970.0,50002,4878437089804288


In [31]:
df_expanded = df_expanded[['orderId', 'id', 'amount']]
df_expanded.rename(columns={'orderId': 'order_id', 'id': 'payment_id'}, inplace=True)
df_expanded.head()

,order_id,payment_id,amount
0,4617000165572608,5000,10203.0
1,4878437089804288,50002,17970.0
2,5124727694426112,1000,6990.0
3,5246704765632512,50004,16280.0
4,5308026161201152,1000,12880.0


In [32]:
# df_expanded.insert(0, 'id', range(1, len(df_expanded) + 1))
df_expanded

,order_id,payment_id,amount
0,4617000165572608,5000,10203.0
1,4878437089804288,50002,17970.0
2,5124727694426112,1000,6990.0
3,5246704765632512,50004,16280.0
4,5308026161201152,1000,12880.0
...,...,...,...
54720,6683520360644608,3000,95458.0
54721,6739510057631744,1000,28787.0
54722,6740702414372864,5103,21990.0
54723,6750384411508736,3000,14503.0


In [33]:
df_expanded.to_csv("./archivos_csv/forma_pago.csv", index=False)

In [64]:
# tabla tipos_forma_pago
forma_pago_parcial['paymentForms'] = forma_pago_parcial['paymentForms'].apply(lambda x: eval(x))

expanded_rows = []
# Iterar sobre cada fila del DataFrame original
for index, row in forma_pago_parcial.iterrows():
    order_id = row['orderId']
    payments = row['paymentForms']
    # Iterar sobre cada producto en la lista de productos
    for payment in payments:
        # Crear un nuevo diccionario que representa una fila expandida
        expanded_row = {**payment, 'orderId': order_id}
        expanded_rows.append(expanded_row)

# Crear un nuevo DataFrame con las filas expandidas
df_expanded = pd.DataFrame(expanded_rows)

df_expanded = df_expanded[['id', 'name']]
df_expanded

C:\Users\matia\AppData\Local\Temp\ipykernel_5800\603849563.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forma_pago_parcial['paymentForms'] = forma_pago_parcial['paymentForms'].apply(lambda x: eval(x))


,id,name
0,5000,Web Pay
1,50002,PedidosYa Efectivo
2,1000,Efectivo
3,50004,Uber eats
4,1000,Efectivo
...,...,...
54953,3000,Tarjeta Débito
54954,1000,Efectivo
54955,5103,UberEats
54956,3000,Tarjeta Débito


In [65]:
df_expanded = df_expanded.drop_duplicates(subset=['id']).reset_index(drop=True)
df_expanded

,id,name
0,5000,Web Pay
1,50002,PedidosYa Efectivo
2,1000,Efectivo
3,50004,Uber eats
4,3000,Tarjeta Débito
5,9001,Transferencia
6,5013,PedidosYa Vouchers
7,5012,PedidosYa
8,5015,Rappi
9,5103,UberEats


In [66]:
df_expanded.to_csv("./archivos_csv/tipos_forma_pago.csv", index=False)

In [34]:
# tabla final de ventas_total
copia.shape

(53604, 18)

In [35]:
copia.drop_duplicates(subset=['orderId'], inplace=True)
copia.shape

(53380, 18)

In [36]:
# la columna paymentId la borro porque no se que significa
ventas_total = copia.drop(columns=['tableName', 'products', 'paymentForms', 'waiterName', 'tableCapaticy', 'paymentId'])
ventas_total.head()

,total,waiterId,gratuity,zoneId,discounts,dateOpen,orderId,subtotal,taxes,payed,tableId,dateClosed
0,6990.0,0,0.0,NaN,0.0,2021-08-14T00:55:44,4617000165572608,5361.0,1629.0,10203.0,V-90002,2021-08-14T01:50:47
1,17970.0,3,0.0,ZB.010,0.0,2021-08-14T01:26:35,4878437089804288,15102.0,2868.0,17970.0,18,2021-08-14T20:23:29
2,6990.0,3,0.0,ZB.010,0.0,2021-08-14T01:33:55,5124727694426112,5874.0,1116.0,6990.0,7,2021-08-14T01:42:23
3,19980.0,3,0.0,ZB.010,-3700.0,2021-08-13T21:10:13,5246704765632512,17381.0,2599.0,16280.0,16,2021-08-13T21:26:44
4,12880.0,3,0.0,ZB.010,0.0,2021-08-13T22:25:19,5308026161201152,10824.0,2056.0,12880.0,3,2021-08-14T00:04:35


In [37]:
ventas_total = ventas_total[['orderId', 'total', 'gratuity', 'discounts', 'taxes', 'subtotal', 'payed', 'tableId', 'waiterId', 'dateOpen', 'dateClosed']]
# ventas_total.rename(columns={"orderId": "id", "tableId": "table_id", "waiterId": "waiter_id", "dateOpen": "date_open", "dateClosed": "date_closed"}, inplace=True)
ventas_total.rename(columns={"tableId": "table_id", "waiterId": "waiter_id", "dateOpen": "date_open", "dateClosed": "date_closed"}, inplace=True)
ventas_total.head()

,orderId,total,gratuity,discounts,taxes,subtotal,payed,table_id,waiter_id,date_open,date_closed
0,4617000165572608,6990.0,0.0,0.0,1629.0,5361.0,10203.0,V-90002,0,2021-08-14T00:55:44,2021-08-14T01:50:47
1,4878437089804288,17970.0,0.0,0.0,2868.0,15102.0,17970.0,18,3,2021-08-14T01:26:35,2021-08-14T20:23:29
2,5124727694426112,6990.0,0.0,0.0,1116.0,5874.0,6990.0,7,3,2021-08-14T01:33:55,2021-08-14T01:42:23
3,5246704765632512,19980.0,0.0,-3700.0,2599.0,17381.0,16280.0,16,3,2021-08-13T21:10:13,2021-08-13T21:26:44
4,5308026161201152,12880.0,0.0,0.0,2056.0,10824.0,12880.0,3,3,2021-08-13T22:25:19,2021-08-14T00:04:35


In [38]:
# tabla orders
orders = ventas_total[['orderId', 'date_open', 'date_closed']]
orders.rename(columns={'orderId': 'id'}, inplace=True)
orders.head()

C:\Users\matia\AppData\Local\Temp\ipykernel_20040\1087992100.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders.rename(columns={'orderId': 'id'}, inplace=True)


,id,date_open,date_closed
0,4617000165572608,2021-08-14T00:55:44,2021-08-14T01:50:47
1,4878437089804288,2021-08-14T01:26:35,2021-08-14T20:23:29
2,5124727694426112,2021-08-14T01:33:55,2021-08-14T01:42:23
3,5246704765632512,2021-08-13T21:10:13,2021-08-13T21:26:44
4,5308026161201152,2021-08-13T22:25:19,2021-08-14T00:04:35


In [39]:
orders.to_csv("./archivos_csv/orders.csv", index=False)

In [40]:
ventas_total = ventas_total.drop(columns=['date_open', 'date_closed'])

In [41]:
ventas_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53380 entries, 0 to 53603
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   orderId    53380 non-null  int64  
 1   total      53380 non-null  float64
 2   gratuity   53380 non-null  float64
 3   discounts  53380 non-null  float64
 4   taxes      53380 non-null  float64
 5   subtotal   53380 non-null  float64
 6   payed      53380 non-null  float64
 7   table_id   53380 non-null  object 
 8   waiter_id  53380 non-null  int64  
dtypes: float64(6), int64(2), object(1)
memory usage: 4.1+ MB


In [42]:
ventas_total.to_csv("./archivos_csv/orders_info.csv", index=False)

In [43]:
# tabla products y hierarchy
para_products_y_hierarchy.head()

,payed,discounts,hierarchyName,netPrice,name,hierarchyId,id,taxes,quantity,orderId
0,6990.0,0.0,Burgers🍔,6990.0,Hamburguesa Americana🥓,AB.410,9050,1116.0,1,4617000165572608
1,3213.0,0.0,Todos los Productos,3213.0,Costo Delivery,AB.,TOTEATDVYCOST,513.0,1,4617000165572608
2,5990.0,0.0,Fries,5990.0,Mexa Grande,AB.010,1090,956.0,1,4878437089804288
3,5990.0,0.0,Fries,5990.0,Cheese and bacon Grande,AB.010,1050,956.0,1,4878437089804288
4,5990.0,0.0,Fries,5990.0,Pollo albahaca grande,AB.010,1920,956.0,1,4878437089804288


In [44]:
products_parcial = para_products_y_hierarchy[['netPrice', 'name', 'hierarchyId', 'id']]
products_parcial.head()

,netPrice,name,hierarchyId,id
0,6990.0,Hamburguesa Americana🥓,AB.410,9050
1,3213.0,Costo Delivery,AB.,TOTEATDVYCOST
2,5990.0,Mexa Grande,AB.010,1090
3,5990.0,Cheese and bacon Grande,AB.010,1050
4,5990.0,Pollo albahaca grande,AB.010,1920


In [45]:
columna_a_mover = products_parcial.pop('id')
products_parcial.insert(0, 'id', columna_a_mover)
products_parcial.head(2)

,id,netPrice,name,hierarchyId
0,9050,6990.0,Hamburguesa Americana🥓,AB.410
1,TOTEATDVYCOST,3213.0,Costo Delivery,AB.


In [46]:
products_parcial.rename(columns={"netPrice": "net_price", "hierarchyId": "hierarchy_id"}, inplace=True)
products = products_parcial[['id', 'name', 'net_price', 'hierarchy_id']]
products.head()

C:\Users\matia\AppData\Local\Temp\ipykernel_20040\1115962810.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products_parcial.rename(columns={"netPrice": "net_price", "hierarchyId": "hierarchy_id"}, inplace=True)


,id,name,net_price,hierarchy_id
0,9050,Hamburguesa Americana🥓,6990.0,AB.410
1,TOTEATDVYCOST,Costo Delivery,3213.0,AB.
2,1090,Mexa Grande,5990.0,AB.010
3,1050,Cheese and bacon Grande,5990.0,AB.010
4,1920,Pollo albahaca grande,5990.0,AB.010


In [47]:
products.drop_duplicates(subset=['id'], inplace=True)
products

C:\Users\matia\AppData\Local\Temp\ipykernel_20040\3041420985.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products.drop_duplicates(subset=['id'], inplace=True)


,id,name,net_price,hierarchy_id
0,9050,Hamburguesa Americana🥓,6990.0,AB.410
1,TOTEATDVYCOST,Costo Delivery,3213.0,AB.
2,1090,Mexa Grande,5990.0,AB.010
3,1050,Cheese and bacon Grande,5990.0,AB.010
4,1920,Pollo albahaca grande,5990.0,AB.010
...,...,...,...,...
216931,7561,tropical gin,8990.0,AB.007
217168,4870,corta corriente,8990.0,AB.007
217431,4880,piscolas,5990.0,AB.007
218466,4890,golden ale,7990.0,AB.007


In [48]:
products

,id,name,net_price,hierarchy_id
0,9050,Hamburguesa Americana🥓,6990.0,AB.410
1,TOTEATDVYCOST,Costo Delivery,3213.0,AB.
2,1090,Mexa Grande,5990.0,AB.010
3,1050,Cheese and bacon Grande,5990.0,AB.010
4,1920,Pollo albahaca grande,5990.0,AB.010
...,...,...,...,...
216931,7561,tropical gin,8990.0,AB.007
217168,4870,corta corriente,8990.0,AB.007
217431,4880,piscolas,5990.0,AB.007
218466,4890,golden ale,7990.0,AB.007


In [49]:
products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 574 entries, 0 to 222508
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            574 non-null    object 
 1   name          574 non-null    object 
 2   net_price     574 non-null    float64
 3   hierarchy_id  574 non-null    object 
dtypes: float64(1), object(3)
memory usage: 22.4+ KB


In [50]:
products.to_csv("./archivos_csv/product.csv", index=False)

In [51]:
hierarchy_parcial = para_products_y_hierarchy[['hierarchyId', 'hierarchyName']]
hierarchy_parcial.head()

,hierarchyId,hierarchyName
0,AB.410,Burgers🍔
1,AB.,Todos los Productos
2,AB.010,Fries
3,AB.010,Fries
4,AB.010,Fries


In [52]:
hierarchy = hierarchy_parcial.drop_duplicates(subset=['hierarchyId']).reset_index(drop=True)
hierarchy

,hierarchyId,hierarchyName
0,AB.410,Burgers🍔
1,AB.,Todos los Productos
2,AB.010,Fries
3,AB.020,Burgers
4,AB.008,Promociones
...,...,...
60,AB.510,ENTRADAS
61,AB.550,VIERNES AUTOR 2*8990
62,AB.150,delivery
63,AB.007,PROMOS PAPA´S SHOP


In [53]:
hierarchy.rename(columns={"hierarchyId": "id", "hierarchyName": "name"}, inplace=True)
hierarchy.head(2)

,id,name
0,AB.410,Burgers🍔
1,AB.,Todos los Productos


In [54]:
hierarchy = hierarchy.drop(8)

In [55]:
hierarchy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64 entries, 0 to 64
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      64 non-null     object
 1   name    64 non-null     object
dtypes: object(2)
memory usage: 1.5+ KB


In [56]:
hierarchy.to_csv("./archivos_csv/hierarchy.csv", index=False)

: 